#  Complex step approximation and Automatic Differentiation
### Francesco Pasqualini
## Complex step approximation

$$  \begin{align}   

{f(x + ih) = f (x) + ih f^\prime(x) - {{h^2 } \over 2!} f^{\prime\prime}(x) + \mathcal{O}(h^{3})\tag{1}  }

\end{align} $$

$$  \begin{align}   

{ f(x) = Re f(x + ih) + \mathcal{O}(h^{2})  } \tag{2}  
\end{align} $$

$$  \begin{align}   

{ f^\prime(x) = Im {f(x + ih)\over h} + \mathcal{O}(h^{2})  } \tag{3}  
\end{align} $$

$$  \begin{align}   

{ f^\prime(x) = Im {f(x + ih)\over h} -  {{h^2 } \over 2!} f^{\prime\prime}(x) + \mathcal{O}(h^{3})  } \tag{4}  
\end{align} $$


>The usual way to approximate derivatives is with finite differences, for example by the forward difference approximation
This approximation has error $\mathcal{O}(h)$ so it is less accurate than the complex step approximation for a given h, but more importantly it is prone to numerical cancellation. For small $h$, $f(x+h)$ and $f(x)$ agree to many significant digits and so in floating-point arithmetic the difference approximation suffers a loss of significant digits. Consequently, as $h$ decreases the error in the computed approximation eventually starts to increase. As numerical analysis textbooks explain, the optimal choice of h that balances truncation error and rounding errors is approximately $h_{opt}$ 


$$  \begin{align}   
{ f^\prime(x) = {{f(x + ih)-f(x)}\over h} + \mathcal{O}(h)  }  \tag{5}
\end{align} $$

$$  
\begin{align}   

{ h_{opt} = 2 { \bigg\vert {u f(x) \over f^{\prime\prime}(x)} \bigg\vert }}^2 \tag{6}
\end{align} $$
>where $u$ is the unit roundoff. The optimal error is therefore of order $u^{1/2}$.

When using the complex-step derivative approximation, in order to effectively
eliminate truncation errors, it is typical to use a step that is many orders of
magnitude smaller than the real part of the calculation. When the truncation errors are eliminated, the higher-order terms of the derivative approximation (4)
are so small that they vanish when added to other terms using finite-precision
arithmetic. We obtain:

$$  \begin{align}   

{f(x + ih) \equiv f (x) + ih f^\prime(x) \tag{7}  }

\end{align} $$

where the imaginary part is exactly the derivative of $f$ times $h$.
The end result
is a sensitivity calculation method that is equivalent to the forward mode of
algorithmic differentiation (also known as automatic differentiation or AD), as observed by Griewank [2000, chap. 10, p. 227].


## Automatic differentiation

Just as the complex step approximation is based on complex numbers, so the automatic differentiation is based on double numbers.

### Dual numbers
Dual numbers are a hypercomplex number system first introduced in the 19th century. They are expressions of the form $a + bε$, where a and b are real numbers, and ε is a symbol taken to satisfy ${\displaystyle \varepsilon ^{2}=0}$.

$${\displaystyle f(a+b\varepsilon )=\sum _{n=0}^{\infty }{\frac {f^{(n)}(a)b^{n}\varepsilon ^{n}}{n!}}=f(a)+bf'(a)\varepsilon }$$

So using Taylor seires we can extend any (analytic) real function to the dual numbers.
The interesting thing is that by computing compositions of these extended functions over the dual numbers and examining the coefficient of ε in the result we find we have automatically computed the derivative of the composition.

### Automatc differentiation

>In mathematics and computer algebra, automatic differentiation
(AD) is a set of techniques to evaluate the derivative of a function specified
by a computer program. AD exploits the fact that every computer program, no
matter how complicated, executes a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.), elementary functions
(exp, log, sin, cos, etc.) and control flow statements. AD takes source code of a
function as input and produces source code of the derived function. By applying
the chain rule repeatedly to these operations, derivatives of arbitrary order can
be computed automatically, accurately to working precision, and using at most
a small constant factor more arithmetic operations than the original program.


## References
[What Is the Complex Step Approximation? - Nick Higham](https://nhigham.com/2020/10/06/what-is-the-complex-step-approximation/)

[The Complex-Step Derivative Approximation - JOAQUIM R. R. A. MARTINS 2003](https://www.researchgate.net/publication/222112601_The_Complex-Step_Derivative_Approximation)

[The complex-step derivative approximation - Joaquim J Martins, Peter Sturdza, Juan J Alonso 2017](https://hal.archives-ouvertes.fr/hal-01483287/document)

[Automatic Differentiation in ROOT - Vassil Vassilev, Aleksandr Efremov, Oksana Shadura 2019](https://www.epj-conferences.org/articles/epjconf/pdf/2020/21/epjconf_chep2020_02015.pdf)





In [ ]:
#include <string>
#include <fstream>
#include "xtl/xbase64.hpp"
#include "nlohmann/json.hpp"
#pragma cling add_include_path("/srv/conda/envs/notebook/include/python3.7m")
#pragma cling add_include_path("/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/include/")
#pragma cling add_library_path("/srv/conda/envs/notebook/lib/")
#pragma cling load("python3.7m")
#include "clad/Differentiator/Differentiator.h"
#include <iostream>
#include "../../matplotlibcpp.h"
#include <iostream>
#include <complex>
#include <cmath>


namespace plt = matplotlibcpp;
namespace im
{
    struct image
    {   
        inline image(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }        
        std::stringstream m_buffer;
    };
    nl::json mime_bundle_repr(const image& i)
    {
        auto bundle = nl::json::object();
        bundle["image/png"] = xtl::base64encode(i.m_buffer.str());
        return bundle;
    }
}

using namespace std;
using namespace std::complex_literals;

typedef complex<long double> dcomp;


long double function_fn(long double x) {
  return exp(x)/(pow((cos(x)),3) + pow(sin(x),3));
}

dcomp function_cfn(dcomp x) {
  return exp(x)/(pow((cos(x)),3) + pow(sin(x),3));
}


double derivative_fn(double x) {
  return (exp(x)*(cos(3*x) + sin(3*x)/2 + (3*sin(x))/2)) / 
            pow(pow(cos(x),3) + pow(sin(x),3),2);
}

long double derivative_finite_diff_approx(long double x,long double h) {
  return (function_fn(x+h) - function_fn(x))/h;
}


long double derivative_complex_step_approx(long double x,long  double h) {
  return imag(function_cfn(dcomp(x,h)))/h;
}


long double derivative_ad_reverse(long double x) {
    long double dy = 0;
    auto f_grad = clad::gradient(function_fn, "x");
    f_grad.execute(x, &dy);
    return dy;
}


    
auto fn_dx = clad::differentiate(function_fn, "x");


In [ ]:

    std::vector<float> x;
    std::vector<float> y;
    std::vector<float> y1;
    double pi;
    double a;
    double b;
    int N;
    N = 1000;
    pi =  2 * asin(1);
    a = - pi/4;
    b = pi / 2;
    for (float j = a; j <= b; j += (b-a)/N) {
        x.push_back(j);
        y.push_back(function_fn(j));
        y1.push_back(fn_dx.execute(j));
    }

plt::plot(x, y);
plt::plot(x, y1);
plt::ylim(0, 6);
plt::xlim(-0.78, 1.7);
plt::save("../../test/test3.png");
auto img = im::image("../../test/test3.png");
img

In [ ]:
    std::vector<double> logspace;
    std::vector<double> error_ff;
    std::vector<double> error_complex_step;
    std::vector<double> error_ad;
double point_x;
point_x =  pi/4;

    for (int i = 15; i >= 1; i -= 1) {
        logspace.push_back(pow(10,-i));
    }

for(double h : logspace) {
   error_ff.push_back(abs(derivative_finite_diff_approx(point_x, h) - derivative_fn(point_x))/abs(derivative_fn(point_x)));
   error_complex_step.push_back(abs(derivative_complex_step_approx(point_x, h) - derivative_fn(point_x))/abs(derivative_fn(point_x)));
   //error_ad.push_back(abs(fn_dx.execute(point_x) - derivative_fn(point_x))/abs(derivative_fn(point_x)));   
   error_ad.push_back(abs(derivative_ad_reverse(point_x) - derivative_fn(point_x))/abs(derivative_fn(point_x)));   
}

plt::figure();
plt::loglog(logspace, error_ff);
plt::loglog(logspace, error_complex_step, "bo");
plt::loglog(logspace, error_ad);
//plt::gca().invert_xaxis();
//plt::xlim(pow(10,-18), 10);
plt::save("../../test/test_err.png");
auto img2 = im::image("../../test/test_err.png");
img2

In [ ]:
error_complex_step

In [ ]:
error_ad

In [ ]:
auto f_grad = clad::gradient(function_fn, "x");
f_grad.dump();

xeus-cling provides a Jupyter kernel for C++ with the help of the C++ interpreter cling and the native implementation of the Jupyter protocol xeus.

Within the xeus-cling framework, Clad can enable automatic differentiation (AD) such that users can automatically generate C++ code for their computation of derivatives of their functions.



## Forward Mode AD

For a function _f_ of several inputs and single (scalar) output, forward mode AD can be used to compute (or, in case of Clad, create a function) computing a directional derivative of _f_ with respect to a single specified input variable. Moreover, the generated derivative function has the same signature as the original function _f_, however its return value is the value of the derivative.

In [2]:
double fn(double x, double y) {
  return x*x*y + y*y;
}

In [3]:
auto fn_dx = clad::differentiate(fn, "x");

In [4]:
fn_dx.execute(5, 3)

30.000000

## Reverse Mode AD

Reverse-mode AD enables the gradient computation within a single pass of the computation graph of _f_ using at most a constant factor (around 4) more arithmetical operations compared to the original function. While its constant factor and memory overhead is higher than that of the forward-mode, it is independent of the number of inputs.

Moreover, the generated function has void return type and same input arguments. The function has an additional argument of type T*, where T is the return type of _f_. This is the “result” argument which has to point to the beginning of the vector where the gradient will be stored.

In [5]:
double fn(double x, double y) {
  return x*x + y*y;
}

In [6]:
auto d_fn_2 = clad::gradient(fn, "x, y");
double d_x, d_y;

In [7]:
d_fn_2.dump();

The code is: void fn_grad(double x, double y, clad::array_ref<double> _d_x, clad::array_ref<double> _d_y) {
    double _t2;
    double _t3;
    double _t4;
    double _t5;
    _t3 = x;
    _t2 = x;
    _t5 = y;
    _t4 = y;
    double fn_return = _t3 * _t2 + _t5 * _t4;
    goto _label0;
  _label0:
    {
        double _r0 = 1 * _t2;
        * _d_x += _r0;
        double _r1 = _t3 * 1;
        * _d_x += _r1;
        double _r2 = 1 * _t4;
        * _d_y += _r2;
        double _r3 = _t5 * 1;
        * _d_y += _r3;
    }
}



## Hessian

In [8]:
double kinetic_energy(double mass, double velocity) {
  return mass * velocity * velocity * 0.5;
}

In [9]:
auto hessian = clad::hessian(kinetic_energy, "mass, velocity");
double matrix[4];
hessian.execute(10, 2, matrix)

In [10]:
matrix

{ 0.0000000, 2.0000000, 2.0000000, 10.000000 }

## Jacobian

In [11]:
void fn_jacobian(double i, double j, double *res) {
  res[0] = i*i;
  res[1] = j*j;
  res[2] = i*j;
}

In [12]:
auto d_fn = clad::jacobian(fn_jacobian);
double res[3] = {0, 0, 0};
double derivatives[6] = {0, 0, 0, 0, 0, 0};
d_fn.execute(3, 5, res, derivatives);

In [13]:
derivatives

{ 6.0000000, 0.0000000, 0.0000000, 10.000000, 5.0000000, 3.0000000 }

In [14]:
std::cout<<"Jacobian matrix:\n";
  for (int i=0; i<3; ++i) {
    for (int j=0; j<2; ++j) {
      std::cout<<derivatives[i*2 + j]<<" ";
    }
    std::cout<<"\n";
  }

Jacobian matrix:
6 0 
0 10 
5 3 


## Functors

In [15]:
class Equation {
  double m_x, m_y;

  public:
  Equation(double x, double y) : m_x(x), m_y(y) {}
  double operator()(double i, double j) {
    return m_x*i*j + m_y*i*j;
  }
  void setX(double x) {
    m_x = x;
  }
};

In [16]:
Equation E(3,5);
auto d_E = clad::differentiate(E, "i");

In [17]:
d_E.dump()

The code is: double operator_call_darg0(double i, double j) {
    double _d_i = 1;
    double _d_j = 0;
    double &_t2 = this->m_x;
    double _t3 = _t2 * i;
    double &_t4 = this->m_y;
    double _t5 = _t4 * i;
    return (0. * i + _t2 * _d_i) * j + _t3 * _d_j + (0. * i + _t4 * _d_i) * j + _t5 * _d_j;
}

